#### Partially verified
Few shot with errors and multi round

In [1]:
import evaluation

evaluation = evaluation.Evaluation.from_config("config_cot.json", "gpt-4")
evaluation.get_few_shot_prompt("append", with_errors=True)

[{'role': 'system',
  'content': 'You are an assistant that given a python program, \nannotates it with appropriate Nagini annotations so that verification succeeds.\n\nNagini is a static verifier for Python. \nOur aim is to given a statically typed Python program, \nto come up with appropriate preconditions (e.g. Requires(is_list(head))), postcondition (e.g. Ensures(is_list(Result()))),\nloop invariants (Invariant(<assertion>)), predicate fold/unfolds (e.g. Fold(is_list(head)) / Unfold(is_list(head))) \nso that the program verifies correctly.\nThe user will provide Python code and the verification errors. \nYou must add or change the specifications so that the resulting code verifies correctly. \nFirst, think step-by-step: explain your reasoning as python code comments e.g.\n# Based on the verification error there might be insufficient permissions ...\n# Hence, I should add an Unfold at line 7 ...\nFollowing the comments, return only the code without any explanation or wrapping. e.g.\

In [2]:
ev = evaluation.run_eval(
    k=1,
    n=1,
    with_errors=True,
)

Running example: prepend ; attempt: 1 ; error depth: 1
Generated program from GPT:
# Based on the verification error, it appears that verification is failing for the essential post condition Ensures(is_list(Result()))
# This can happen due to lack of folding the predicate for the newly created node 'n' before returning it. 

def prepend(head: Node, val: int) -> Node:
    """Prepends a new node with the given value to the list."""
    Requires(is_list(head))
    Ensures(is_list(Result()))
    n = Node(val, head)
    Fold(is_list(n))
    return n

response ['', 'Verification successful', 'Verification took 1.99 seconds.']
Verification result:
 Verification successful 


Running example: append ; attempt: 1 ; error depth: 1
Generated program from GPT:
# The error message indicates that there is insufficient permission to access head.next. 
# This is because the function tries to access head.next without unfolding the predicate is_list(head).
# Hence, I should add an Unfold before trying t

In [3]:
ev

EvalResult(results={'prepend': True, 'append': True, 'find': False, 'find_iter': True, 'remove': True, 'join_lists': True, 'reverse': True}, verified_at={'prepend': (1, 1), 'append': (1, 1), 'find_iter': (1, 1), 'remove': (1, 1), 'join_lists': (1, 1), 'reverse': (1, 1)})

In [4]:
ev #without cot

EvalResult(results={'prepend': False, 'append': True, 'find': True, 'find_iter': True, 'remove': False, 'join_lists': True, 'reverse': False}, verified_at={'append': (1, 1), 'find': (1, 1), 'find_iter': (1, 1), 'join_lists': (1, 1)})

In [5]:
# print contents of the cached verification result
import pickle

with open("nagini_examples/verif_result_partial.pkl", "rb") as f:
    print(pickle.load(f))

{'prepend': 'Verification failed: Postcondition of prepend might not hold. There might be insufficient permission to access is_list(Result()). at line 4.12', 'append': 'Verification failed: Conditional statement might fail. There might be insufficient permission to access head.next. at line 5.7', 'find': 'Verification failed: Method call might fail. There might be insufficient permission to access head.next. at line 9.13', 'find_iter': 'Verification failed: Conditional statement might fail. There might be insufficient permission to access ptr.val. at line 9.11', 'remove': 'Verification failed: Postcondition of remove might not hold. There might be insufficient permission to access is_list(Result()). at line 4.12', 'join_lists': 'Verification failed: Method call might fail. There might be insufficient permission to access head1.next. at line 10.17', 'reverse': 'Verification failed: Unfold might fail. There might be insufficient permission to access is_list(ptr). at line 15.8'}


In [19]:
reverse_prompt = evaluation.get_few_shot_prompt("reverse", with_errors=True)
reverse_prompt[-1]['content'] += "\nIMPORTANT: THINK STEP BY STEP AND REMEMBER TO EXPLAIN YOUR WORK IN PYTHON COMMENTS BEFORE PRODUCING THE VERIFYING PROGRAM."
reverse_prompt

[{'role': 'system',
  'content': 'You are an assistant that given a python program, \nannotates it with appropriate Nagini annotations so that verification succeeds.\n\nNagini is a static verifier for Python. \nOur aim is to given a statically typed Python program, \nto come up with appropriate preconditions (e.g. Requires(is_list(head))), postcondition (e.g. Ensures(is_list(Result()))),\nloop invariants (Invariant(<assertion>)), predicate fold/unfolds (e.g. Fold(is_list(head)) / Unfold(is_list(head))) \nso that the program verifies correctly.\nThe user will provide Python code and the verification errors. \nYou must add or change the specifications so that the resulting code verifies correctly. \nFirst, think step-by-step: explain your reasoning as python code comments e.g.\n# Based on the verification error there might be insufficient permissions ...\n# Hence, I should add an Unfold at line 7 ...\nFollowing the comments, return only the code without any explanation or wrapping. e.g.\

In [22]:
from evaluation import _call_gpt_with_timeout, _print_and_process_response
response = _call_gpt_with_timeout(
    model="gpt-3.5-turbo",
    messages=evaluation.get_few_shot_prompt("prepend", with_errors=True),
)
print(response)
program_snippet = _print_and_process_response(response)
print("Verification result:\n", evaluation._verify_program_snippet("list", program_snippet))

{
  "id": "chatcmpl-8IO7cYPlYpUuiRYtdIaboFzIrM6zB",
  "object": "chat.completion",
  "created": 1699393320,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "def prepend(head: Node, val: int) -> Node:\n    \"\"\"Prepends a new node with the given value to the list.\"\"\"\n    Requires(is_list(head))\n    Ensures(is_list(Result()))\n    Fold(is_list(head))\n    n = Node(val, head)\n    Unfold(is_list(n))\n    return n"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 2033,
    "completion_tokens": 66,
    "total_tokens": 2099
  }
}
Generated program from GPT:
def prepend(head: Node, val: int) -> Node:
    """Prepends a new node with the given value to the list."""
    Requires(is_list(head))
    Ensures(is_list(Result()))
    Fold(is_list(head))
    n = Node(val, head)
    Unfold(is_list(n))
    return n
response ['', 'Verification failed', 'Errors:', 'Fold might 